<a href="https://colab.research.google.com/github/kyoungjini/Korean-Sentiment-Analyzer/blob/main/kobert_korean_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoBERT 모델 중 tokenizer만 가져오기

In [1]:
!pip install 'git+https://git@github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://****@github.com/SKTBrain/KoBERT.git to /tmp/pip-install-a0n_vgt5/kobert-tokenizer_60bafc66c4b34ad98632434bd8c45749
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-install-a0n_vgt5/kobert-tokenizer_60bafc66c4b34ad98632434bd8c45749
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 5c46b1c68e4755b54879431bd302db621f4d2f47
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=edfb7cb962dbaa31236cb5b6eb01e3647b7a4c9adc8bc1c2eb14bb2a261e4227
  Stored in directory: /tmp/pip-ephem-wheel-cache-jixp8ygu/wheels/db/6c/61/d61614a2a1d912797e52c01a97f5becb8ba88e14435e75ee5c
Successfully built kobert_tokenizer


# Colab 환경 설정 (패키지 설치)

In [2]:
!pip install boto3==1.15.18
!pip install gluonnlp==0.8.0
!pip install mxnet==1.7.0.post2
!pip install onnxruntime
!pip install sentencepiece==0.1.96
!pip install torch
!pip install transformers==4.8.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentry-sdk 2.18.0 requires urllib3>=1.26.11, but you have urllib3 1.25.11 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0

# 내 구글 드라이브와 연결하기

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 필요한 라이브러리 import

In [4]:
import os
import json

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

from kobert_tokenizer import KoBERTTokenizer

from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# GPU 설정

In [5]:
# device = torch.device("cuda:0")
# device

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

!nvidia-smi

Thu Nov 21 00:34:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# 난수 고정

In [6]:
random_seed = 42
import random

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

# 데이터셋 전처리1 - 감성대화말뭉치 데이터

In [7]:
### 감성대화말뭉치 데이터 전처리
df1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/jeju-voc-analyze/감성대화말뭉치(최종데이터)_Training.xlsx')
#df1.head()

# 감정 매핑
emotion_mapping = {"기쁨": 1, "분노": 0, "불안": 0, "상처": 0, "당황": 0, "슬픔": 0 }
df1['감정_대분류'] = df1['감정_대분류'].map(emotion_mapping)

# 데이터셋 정리
data_list1 = [[str(sentence), int(emotion)] for sentence, emotion in zip(df1['사람문장1'], df1['감정_대분류'])]
#data_list1

# 데이터셋 전처리2 - 속성기반 감정분석 데이터_샘플

In [8]:
### 속성기반 감정분석 데이터 샘플 전처리
base_folder_path = '/content/drive/MyDrive/Colab Notebooks/jeju-voc-analyze/속성기반 감정분석 데이터(Sample)_Training_라벨링데이터/SNS'

target_subcategory = '05. 생활'

# 생활 카테고리 내 모든 파일 경로 수집
json_files = []
target_path = os.path.join(base_folder_path, target_subcategory)
for file in os.listdir(target_path):
    if file.endswith('.json'):
        json_files.append(os.path.join(target_path, file))

# JSON 파일 내 데이터 수집
data = []
for file_path in json_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)
        for item in json_data:
            if 'Aspects' in item:
              for aspect in item['Aspects']:
                data.append({
                    "SentimentText": aspect['SentimentText'],
                    "SentimentPolarity": aspect['SentimentPolarity']
                })

df2 = pd.DataFrame(data)

# '중립' 감정 상태인 데이터 제외시키기
df2_filtered = df2[df2['SentimentPolarity'] != '0'].copy()

# '부정' 감정 상태를 -1에서 0으로 변경하기
df2_filtered.loc[df2_filtered['SentimentPolarity'] == '-1', 'SentimentPolarity'] = '0'

#from google.colab import data_table
#data_table.DataTable(df2)

# 데이터셋 정리
data_list2 = [[str(sentence), int(emotion)] for sentence, emotion in zip(df2_filtered['SentimentText'], df2_filtered['SentimentPolarity'])]
# print(len(data_list2))
# data_list2

# 데이터셋 전처리3 - 두 데이터 리스트 합치기

In [9]:
data_list = data_list1 + data_list2
data_list[0]

['일은 왜 해도 해도 끝이 없을까? 화가 난다.', 0]

# 데이터 분리 (Train & Test)

In [10]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=random_seed)

print(len(dataset_train))
print(len(dataset_test))

65986
16497


# 데이터 전처리1 - 토큰화, 정수 인코딩, 패딩

In [11]:
### Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

class BERTSentenceTransform:
    def __init__(self, tokenizer, max_seq_length, vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        # 입력 문장을 BERT 토크나이저로 토큰화
        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer.tokenize(text_b)

        # 문장 길이 조정 (초과 시 잘라냄)
        if tokens_b:
            self._truncate_seq_pair(tokens_a, tokens_b, self._max_seq_length - 3)
        else:
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # BERT가 요구하는 특별 토큰([CLS], [SEP])을 추가하고, 토큰을 ID로 변환함
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)  # 시작 토큼
        tokens.extend(tokens_a)         # 첫 번째 문장
        tokens.append(vocab.sep_token)  # 분리 토큰
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        valid_length = len(input_ids)

        # 패딩 추가 (짧은 문장을 max_seq_length에 맞추기 위함)
        if self._pad:
            padding_length = self._max_seq_length - valid_length
            # input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            input_ids.extend([self._tokenizer.pad_token_id] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'), np.array(segment_ids, dtype='int32')

### BERT 학습에 사용할 데이터셋 정의
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):

        # 데이터셋의 각 문장을 BERTSentenceTransform을 사용해 전처리
        transform = BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        # 문장 인덱스와 레이블 인덱스를 활용해 입력과 레이블 생성
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        # 데이터셋에서 샘플을 가져올 때 문장과 레이블을 반환
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        # 데이터셋 크기 반환
        return (len(self.labels))

# !ls -> 현재 디렉토리 기준으로 실행됨
#!ls /content/drive/MyDrive

# 데이터 전처리2 - KoBERT 토크나이저 불러오기

In [12]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


# 데이터 전처리3 - KoBERT 모델과 사전 불러오기

In [13]:
def get_kobert_model(model_path, ctx="cpu"):
  bertmodel = BertModel.from_pretrained(model_path)
  device = torch.device(ctx)
  bertmodel.to(device)
  bertmodel.eval()
  # vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file, padding_token='[PAD]')
  # return bertmodel, vocab_b_obj
  return bertmodel, tokenizer

# KoBERT 모델과 사전 불러오기
# bertmodel, vocab = get_kobert_model('skt/kobert-base-v1', tokenizer.vocab_file)
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1')

data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


# KoBERT 학습 모델 불러오기

In [14]:
### 긍정/부정 클래스로 분류할 수 있음
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        # _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        # if self.dr_rate:
        #     out = self.dropout(pooler)
        # return self.classifier(out)

        # Get the output from the KoBERT model
        outputs = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(), attention_mask=attention_mask.float().to(token_ids.device))

        # Access the pooler_output (or last_hidden_state if needed)
        pooler_output = outputs.pooler_output

        if self.dr_rate:
            out = self.dropout(pooler_output)
        else:
            out = pooler_output

        return self.classifier(pooler_output)

### 정의한 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

# Prepare optiimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# 모델 학습시키기

In [15]:
from os.path import isfile

# Checkpoint 저장 및 로드 함수 정의
def save_checkpoint(epoch, model, optimizer, scheduler, train_history, test_history, loss_history, checkpoint_path="checkpoint.pth"):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'train_history': train_history,
        'test_history': test_history,
        'loss_history': loss_history
    }, checkpoint_path)
    print(f"Checkpoint saved at epoch {epoch + 1}")

def load_checkpoint(model, optimizer, scheduler, checkpoint_path="checkpoint.pth"):
    if os.path.isfile(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        train_history = checkpoint['train_history']
        test_history = checkpoint['test_history']
        loss_history = checkpoint['loss_history']
        print(f"Checkpoint loaded. Resuming from epoch {start_epoch}")
        return start_epoch, train_history, test_history, loss_history
    else:
        print("No checkpoint found. Starting from scratch.")
        return 0, [], [], []

# 시작 지점 설정
start_epoch, train_history, test_history, loss_history = load_checkpoint(model, optimizer, scheduler)

for e in range(start_epoch, num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    torch.no_grad()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

    # Checkpoint 저장
    save_checkpoint(e, model, optimizer, scheduler, train_history, test_history, loss_history)

No checkpoint found. Starting from scratch.


<ipython-input-15-990c3a7f72aa>:39: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1032 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.6903769969940186 train acc 0.484375
epoch 1 batch id 201 loss 0.1300922930240631 train acc 0.8756218905472637
epoch 1 batch id 401 loss 0.21700894832611084 train acc 0.9141599127182045
epoch 1 batch id 601 loss 0.31184324622154236 train acc 0.9281146006655574
epoch 1 batch id 801 loss 0.14102163910865784 train acc 0.9361735330836455
epoch 1 batch id 1001 loss 0.024292169138789177 train acc 0.9419486763236763
epoch 1 train acc 0.9424509447674418


<ipython-input-15-990c3a7f72aa>:60: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


...

epoch 4 test acc 0.9759383404524601
Checkpoint saved at epoch 4


  0%|          | 0/1032 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.0025078749749809504 train acc 1.0
epoch 5 batch id 201 loss 0.00525495084002614 train acc 0.9946361940298507
epoch 5 batch id 401 loss 0.09011640399694443 train acc 0.994661783042394
epoch 5 batch id 601 loss 0.006819829810410738 train acc 0.9950343178036606
epoch 5 batch id 801 loss 0.004327455535531044 train acc 0.995357365792759
epoch 5 batch id 1001 loss 0.001016204128973186 train acc 0.9955825424575424
epoch 5 train acc 0.995594113372093


  0%|          | 0/258 [00:00<?, ?it/s]

epoch 5 test acc 0.9751930568739124
Checkpoint saved at epoch 5


# 결과물 테스트

In [19]:
tok = tokenizer

def predict(predict_sentence):

  data = [predict_sentence, '0']
  dataset_another = [data]

  another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)

      valid_length= valid_length
      label = label.long().to(device)

      out = model(token_ids, valid_length, segment_ids)

      test_eval=[]
      for i in out:
          logits=i
          logits = logits.detach().cpu().numpy()

          if np.argmax(logits) == 0:
              test_eval.append("부정이")
          elif np.argmax(logits) == 1:
              test_eval.append("긍정이")

      print(">> " + test_eval[0] + " 느껴집니다.")

end = 1
while end == 1 :
  sentence = input("하고싶은 말을 입력해주세요 : ")
  if sentence == 0 :
    break
  predict(sentence)
  print("\n")

하고싶은 말을 입력해주세요 : 제조일이 5개월 지난걸 배송... 다시는 삼다수 안먹음
>> 부정이 느껴집니다.


하고싶은 말을 입력해주세요 : 배송이 너무 늦어서 다음번엔 안 살 거예요
>> 부정이 느껴집니다.


하고싶은 말을 입력해주세요 : 세일할 때 사면 안된다는걸 알게되었음 배송이 너무 늦음
>> 부정이 느껴집니다.


하고싶은 말을 입력해주세요 : 엘리베이터 없는 5층이라 미안은 하지만 예의가 아니네요
>> 긍정이 느껴집니다.


하고싶은 말을 입력해주세요 : 도착보장 상품인데도 배송이 늦고 엉망
>> 부정이 느껴집니다.


하고싶은 말을 입력해주세요 : 
>> 긍정이 느껴집니다.




KeyboardInterrupt: Interrupted by user

# 결과 파일 저장

In [25]:
save_dir = '/content/drive/MyDrive/trained_models'
os.makedirs(save_dir, exist_ok=True)
model_path = os.path.join(save_dir, 'kobert_weightedloss.pt')
torch.save(model.state_dict(), model_path)
print(f"모델이 {model_path}에 저장되었습니다.")

#print(os.path.exists('/content/drive/MyDrive/trained_models'))
#!ls -lh /content/drive/MyDrive/trained_models
#!find /content/drive/MyDrive -name 'kobert_weightedloss.pt'

모델이 /content/drive/MyDrive/trained_models/kobert_weightedloss.pt에 저장되었습니다.
True
total 352M
-rw------- 1 root root 352M Nov 21 02:30 kobert_weightedloss.pt
/content/drive/MyDrive/trained_models/kobert_weightedloss.pt
